In [8]:
import quantumpropagator as qp
import matplotlib.pyplot as plt
import matplotlib
from ipywidgets import interact,fixed #, interactive, fixed, interact_manual
import ipywidgets as widgets

import os
import sys
import glob
import pandas as pd
import numpy as np

%matplotlib ipympl

plt.rcParams.update({'font.size': 8})

In [9]:
def read_dipole_file(dipole_file_path):
    nstates = 8
    permanents = []
    transitions = []

    for lab1 in ['x','y','z']:
        for lab2 in range(nstates):
            permanents.append('perm_{}_{}'.format(lab1,lab2))
            for lab3 in range(lab2+1,nstates):
                transitions.append('trans_{}_{}_{}'.format(lab1,lab2,lab3))
                
    all_labels = ['fs_2','dipx','dipy','dipz'] +  permanents + transitions
    df_dipo_pump = pd.read_csv(dipole_file_path, delim_whitespace=True, names=all_labels)
    return(df_dipo_pump)

def copy_paste_output(pump_df,probe_df, out_name):
    '''
    This function takes a pump and a probe run and returns a new dataframe with pump_probe pasted together
    '''
    fs_cut = probe_df['fs'].iloc[0]
    # the index on the pump sheet is where the first fs value of the probe is.
    index_cut = pump_df.index[pump_df['fs'] == fs_cut][0]
    pump_cut = pump_df.iloc[:index_cut]
    final_paper = pd.concat((pump_cut,probe_df))
    final_paper.to_csv(out_name)
    return(final_paper)

def copy_paste_dipos(pump_df_d,probe_df_d, out_name):
    '''
    This function takes a pump and a probe run and returns a new dataframe with pump_probe pasted together
    '''
    fs_cut = probe_df_d['fs_2'].iloc[0]
    # the index on the pump sheet is where the first fs value of the probe is.
    index_cut = pump_df_d.index[pump_df_d['fs_2'] == fs_cut][0]
    pump_cut = pump_df_d.iloc[:index_cut]
    final_paper = pd.concat((pump_cut,probe_df_d))
    final_paper.to_csv(out_name)
    return(final_paper)

def pythonfft(signal_time,signal,pad_length):
    aaa = np.fft.fft(np.pad(signal,(0,pad_length)))
    time_au = qp.fromFsToAu(signal_time)
    dt = time_au[1]- time_au[0]
    bb = np.fft.fftfreq(time_au.size + pad_length)

    time_au_max = np.amax(time_au)
    bbb = qp.fromHartoEv(bb*2*np.pi/dt)
    indexes = np.where(bb>=0)
    return(bbb[indexes],aaa[indexes])


In [10]:
root = '/home/alessio/w-August-Run/'
pump = 'b-UV-0.22_0000'
probe = 'y-probe-UV_0000'

fol_pump = os.path.join(root, pump)
fol_probe = os.path.join(root, probe)
output_norm_pump = os.path.join(fol_pump, 'output')
output_dipo_pump = os.path.join(fol_pump, 'Output_Dipole')
output_norm_probe = os.path.join(fol_probe, 'output')
output_dipo_probe = os.path.join(fol_probe, 'Output_Dipole')

df_norm2_pump = pd.read_csv(output_norm_pump, delim_whitespace=True, index_col=0, names=['counter', 'steps', 'fs','Norm deviation','Kinetic','Potential','Total','Total Deviation','Xpulse','Ypulse','Zpulse','AbZino'])
df_norm2_probe = pd.read_csv(output_norm_probe, delim_whitespace=True, index_col=0, names=['counter', 'steps', 'fs','Norm deviation','Kinetic','Potential','Total','Total Deviation','Xpulse','Ypulse','Zpulse','AbZino'])

df_dipo_pump = read_dipole_file(output_dipo_pump)
df_dipo_probe = read_dipole_file(output_dipo_probe)

ppname = os.path.join('/home/alessio/w-August-Run/','{}_{}.csv'.format(pump,probe))
ppname_dip = os.path.join('/home/alessio/w-August-Run/','{}_{}_dipole.csv'.format(pump,probe))

df1 = copy_paste_output(df_norm2_pump, df_norm2_probe, ppname)
dip1 = copy_paste_dipos(df_dipo_pump, df_dipo_probe, ppname_dip)

In [11]:
# cut the shortest, please
(df1.shape, dip1.shape)
num1, _ = df1.shape
num2, _ = dip1.shape
cut_here = np.amin([num1,num2])
df = df1[:cut_here]
dip = dip1[:cut_here]

In [12]:
x = df['fs']

pulsex = df['Xpulse']
pulsey = df['Ypulse']
pulsez = df['Zpulse']

together = np.stack((pulsex,pulsey,pulsez), axis=1)
y = np.sum(together,axis=1)

fig, ax0 = plt.subplots(1,1,figsize = (10,6))
ax0.plot(x,y)
ax0.set_xlabel('fs')
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
pad_length = 10000

freq, signal_fft = pythonfft(np.array(x),np.array(y),pad_length)
fig, ax0 = plt.subplots(1,1,figsize = (10,6))
#ax0.plot(freq, signal_fft)
ax0.plot(freq, np.real(signal_fft))
ax0.plot(freq, np.imag(signal_fft))
ax0.set_xlim(0,10)
ax0.set_xlabel('eV')

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
plt.close('all')

pad_length = 10000

xx = dip['fs_2']
x = df['fs']

#for ylabel in ['dipx', 'dipy', 'dipz','trans_x_0_1']:
for ylabel in ['dipx', 'dipy', 'dipz', 'trans_x_0_1', 'trans_z_0_1']: # threshold component also.
    pulsex = df['Xpulse']
    pulsey = df['Ypulse']
    pulsez = df['Zpulse']

    together = np.stack((pulsex,pulsey,pulsez), axis=1)
    y = np.sum(together,axis=1)
    
    yy = dip[ylabel]

    freq_dip, signal_fft_dip = pythonfft(np.array(xx),np.array(yy),pad_length)

    fig, [[ax0, ax1], [ax2, ax3], [ax4, ax5]] = plt.subplots(3,2,figsize = (10,6))
    #ax0.plot(freq_dip, signal_fft_dip)

        
    ax0.plot(xx,yy)
    ax0.set_xlim(0,100)
    ax0.set_xlabel('fs')
    ax0.set_title('{} - signal'.format(ylabel))

    ax2.plot(freq, np.real(signal_fft_dip))
    ax2.plot(freq, np.imag(signal_fft_dip))
    ax2.set_xlim(0,10)
    ax2.set_xlabel('eV')
    ax2.set_title('{} - FT'.format(ylabel))
    
    ax1.plot(x,y)
    ax1.set_xlabel('fs')
    ax1.set_title('{} - Pulse'.format(ylabel))
    
    freq, signal_fft = pythonfft(np.array(x),np.array(y),pad_length)
    ax3.plot(freq, np.real(signal_fft))
    ax3.plot(freq, np.imag(signal_fft))
    ax3.set_xlim(0,10)
    ax3.set_xlabel('eV')
    ax3.set_title('{} - Pulse FFT'.format(ylabel))
    
    S_W = -2 * np.imag(np.conj(signal_fft) * signal_fft_dip)
    ax4.plot(freq_dip, S_W)
    ax4.set_xlim(0,10)
    ax4.set_xlabel('eV')
    ax4.set_title('{} - Transient Absorption Spectra'.format(ylabel))

    #fig.suptitle(ylabel)
    fig.canvas.layout.height = '900px'
    fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# graph and save the big components

In [15]:
nstates = 8
permanents = []
transitions = []

for lab1 in ['x','y','z']:
    for lab2 in range(nstates):
        permanents.append('perm_{}_{}'.format(lab1,lab2))
        for lab3 in range(lab2+1,nstates):
            transitions.append('trans_{}_{}_{}'.format(lab1,lab2,lab3))
                
for cart in ['x','y','z']:
    timefs = dip['fs_2']
    time = qp.fromFsToAu(timefs)
    this_full_dipole = 'dip{}'.format(cart)
    tot_signal = dip[this_full_dipole]
    full_list_label = permanents + transitions
    full_list_this_cart = [x for x in full_list_label if cart in x]
    threshold = np.linalg.norm(tot_signal)*0.05 # 5% of the norm
    big_components = [ x for x in full_list_this_cart if np.linalg.norm(dip[x]) > threshold or x in ['trans_x_0_1', 'trans_y_0_1', 'trans_z_0_1'] ] # the one which contributes more than 5% to the norm
    print(big_components)

['trans_x_0_1', 'trans_x_1_2', 'trans_x_1_3', 'trans_x_1_4']
['trans_y_0_1', 'trans_y_1_2', 'trans_y_1_3', 'trans_y_1_4', 'trans_y_1_5', 'trans_y_2_3', 'trans_y_6_7']
['perm_z_0', 'perm_z_1', 'trans_z_0_1', 'trans_z_1_4']
